In [1]:
# pip install yfinance pandas numpy matplotlib lxml requests
import datetime as dt
import io
import math
import re
import sys
from typing import List

import numpy as np
import pandas as pd
import requests
import yfinance as yf
import matplotlib.pyplot as plt

# --- 1) Get current S&P 500 tickers from Wikipedia ---------------------------
def get_sp500_tickers() -> List[str]:
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    # pandas.read_html can parse the table directly
    tables = pd.read_html(url, flavor="lxml")
    # The first table is the constituents table on Wikipedia
    df = tables[0]
    tickers = df["Symbol"].astype(str).tolist()
    # Yahoo uses '-' instead of '.' (e.g., BRK.B -> BRK-B, BF.B -> BF-B)
    tickers = [t.replace(".", "-").strip() for t in tickers]
    # Deduplicate and keep simple sanity filters
    tickers = sorted(set([t for t in tickers if re.fullmatch(r"[A-Z0-9\-]+", t)]))
    return tickers

# --- 2) Download daily data (Open, High) for ~1y -----------------------------
def fetch_ohlc(tickers: List[str], start: str, end: str) -> pd.DataFrame:
    """
    Returns a DataFrame with MultiIndex columns (Field, Ticker) when group_by='column'.
    We'll request in batches to be gentle and reduce timeouts.
    """
    all_data = []
    batch_size = 60  # yfinance handles threading internally
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i+batch_size]
        df = yf.download(
            tickers=batch,
            start=start,
            end=end,
            interval="1d",
            auto_adjust=False,
            group_by="column",
            threads=True,
            progress=False,
        )
        # Ensure 2-level columns: (Field, Ticker)
        if isinstance(df.columns, pd.MultiIndex):
            all_data.append(df.loc[:, ["Open", "High"]])
        else:
            # Single-ticker edge case: add ticker level
            df2 = pd.concat(
                {batch[0]: df[["Open", "High"]]},
                axis=1
            ).swaplevel(axis=1)
            all_data.append(df2)
    if not all_data:
        raise RuntimeError("No data downloaded. Check network or tickers.")
    data = pd.concat(all_data, axis=1).sort_index(axis=1)
    # Reorder columns to (Field, Ticker)
    data = data.reindex(["Open", "High"], axis=1, level=0)
    return data

# --- 3) Compute daily change rate: (High / Open) - 1 -------------------------
def compute_change_rates(ohl: pd.DataFrame) -> pd.Series:
    """
    ohl: MultiIndex columns (Field, Ticker), index = DatetimeIndex
    Returns a single Series of all (High/Open - 1) values stacked across tickers & days.
    """
    open_df = ohl["Open"]
    high_df = ohl["High"]
    with np.errstate(divide="ignore", invalid="ignore"):
        rates = (high_df / open_df) - 1.0
    # Drop invalid/zero-open rows
    rates = rates.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    # Flatten to long vector
    long = rates.stack(dropna=True)  # index: (date, ticker)
    # By definition High >= Open; negative values could occur only if data glitches exist—clip them.
    long = long.clip(lower=0)
    return long.rename("change_rate")

# --- 4) Bin into [0%,1%), [1%,2%), ... with overflow -------------------------
def bin_and_count(values: pd.Series, max_pct_cap: int | None = None) -> pd.DataFrame:
    max_observed_pct = int(math.ceil(values.max() * 100.0))
    # cap the upper bound to keep the histogram readable
    if max_pct_cap is None:
        # Most days will be < 10% from open to high; still, be safe up to 30%
        max_pct_cap = max(10, min(30, max_observed_pct))
    edges = np.arange(0, max_pct_cap + 1) / 100.0  # e.g., 0.00, 0.01, ..., 0.30
    # Create an overflow bin for >= last edge
    counts, _ = np.histogram(values, bins=np.append(edges, np.inf))
    labels = [f"[{i}%,{i+1}%)" for i in range(0, max_pct_cap)] + [f">={max_pct_cap}%"]
    out = pd.DataFrame({"bin": labels, "count": counts})
    out["share"] = out["count"] / counts.sum()
    return out

# --- 5) Main -----------------------------------------------------------------
def main():
    # Choose a recent 365-day window (end yesterday to avoid partial current day)
    tz = dt.timezone(dt.timedelta(hours=-4))  # Toronto (EDT) summer offset; not critical here
    end_date = dt.datetime.now(tz).date() - dt.timedelta(days=1)
    start_date = end_date - dt.timedelta(days=365)

    print("Fetching S&P 500 tickers from Wikipedia...")
    tickers = get_sp500_tickers()
    print(f"Tickers found: {len(tickers)}")

    print(f"Downloading daily data from {start_date} to {end_date} ...")
    ohl = fetch_ohlc(tickers, start=start_date.isoformat(), end=end_date.isoformat())

    print("Computing (High / Open) - 1 ...")
    all_rates = compute_change_rates(ohl)
    n_days = len(pd.bdate_range(start_date, end_date))  # trading days proxy
    print(f"Data points (stock-days) collected: {len(all_rates):,} "
          f"(expected up to ~{len(tickers)*n_days:,})")

    print("Binning into 1% buckets from 0% upward (with overflow)...")
    table = bin_and_count(all_rates, max_pct_cap=None)
    print("\nHistogram counts:")
    print(table.to_string(index=False, formatters={"share": "{:.2%}".format}))

    # --- Plot (simple matplotlib histogram of the raw values) -----------------
    # Use explicit bin edges matching the table, plus overflow
    max_bin = int(math.ceil(all_rates.max() * 100.0))
    max_bin = max(10, min(30, max_bin))  # keep readable; change if you want
    edges = np.append(np.arange(0, max_bin + 1) / 100.0, np.inf)

    plt.figure(figsize=(9, 5.5))
    plt.hist(all_rates.values, bins=edges)
    plt.title("S&P 500 (current constituents): Distribution of Daily (High / Open − 1)\nLast ~365 days")
    plt.xlabel("Intraday upside from Open to High (fraction)")
    plt.ylabel("Count of stock-days")
    # Tick labels as percents up to the cap
    xticks = np.arange(0, max_bin + 1) / 100.0
    plt.xticks(xticks, [f"{int(x*100)}%" for x in xticks], rotation=0)
    plt.tight_layout()
    plt.show()

# if __name__ == "__main__":
#     try:
#         main()
#     except Exception as e:
#         print("ERROR:", e, file=sys.stderr)
#         print("\nTips:\n"
#               "• Make sure you have internet access for Wikipedia and Yahoo Finance.\n"
#               "• If Wikipedia blocks read_html, replace get_sp500_tickers() with a static CSV of tickers.\n"
#               "• If some tickers fail, lower the batch_size or re-run; yfinance can be flaky.\n")


In [2]:
# requirements:
#   pip install pandas yfinance requests lxml

import pandas as pd
import numpy as np
import yfinance as yf

WIKI_SP500_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def load_sp500_constituents() -> pd.DataFrame:
    """
    Returns DataFrame with columns: symbol, company_name, yahoo_symbol
    - 'symbol' is canonical S&P 500 ticker (e.g., 'BRK.B')
    - 'yahoo_symbol' is Yahoo Finance format (e.g., 'BRK-B')
    """
    tbl = pd.read_html(WIKI_SP500_URL, flavor="lxml")[0]
    tbl = tbl.rename(columns={"Symbol": "symbol", "Security": "company_name"})
    tbl = tbl[["symbol", "company_name"]].copy()
    tbl["yahoo_symbol"] = tbl["symbol"].str.replace(".", "-", regex=False)
    return tbl

def download_sp500_ohlc(spx_df: pd.DataFrame, period_days: int = 220, show_progress: bool = False) -> pd.DataFrame:
    """
    Download daily OHLC (we keep Open/High) for all S&P 500 tickers over ~period_days.
    Returns a LONG DataFrame with columns:
        date (datetime64[ns]), symbol, company_name, Open, High

    Tip: choose period_days >= lookback_days + buffer (e.g., +60).
    """
    tickers = spx_df["yahoo_symbol"].tolist()

    data = yf.download(
        tickers=tickers,
        period=f"{period_days}d",
        interval="1d",
        group_by="ticker",
        auto_adjust=False,
        threads=True,
        progress=show_progress,
    )

    records = []
    # Mapping back to canonical symbols and names
    name_map = dict(zip(spx_df["yahoo_symbol"], spx_df["company_name"]))
    sym_map  = dict(zip(spx_df["yahoo_symbol"], spx_df["symbol"]))

    if isinstance(data.columns, pd.MultiIndex):
        # (ticker, field)
        for ysym in tickers:
            cols_needed = [(ysym, "Open"), (ysym, "High")]
            if all(c in data.columns for c in cols_needed):
                sub = data.loc[:, cols_needed].dropna()
                if sub.empty:
                    continue
                sub.columns = ["Open", "High"]
                sub = sub[(sub["Open"] > 0) & sub["High"].notna()]
                for dt, row in sub.iterrows():
                    records.append({
                        "date": pd.to_datetime(dt),
                        "symbol": sym_map.get(ysym, ysym),
                        "company_name": name_map.get(ysym, ""),
                        "Open": float(row["Open"]),
                        "High": float(row["High"]),
                    })
    else:
        # Single-ticker fallback (unlikely for the whole index)
        if set(["Open", "High"]).issubset(set(data.columns)):
            ysym = tickers[0]
            sub = data[["Open", "High"]].dropna()
            sub = sub[(sub["Open"] > 0) & sub["High"].notna()]
            for dt, row in sub.iterrows():
                records.append({
                    "date": pd.to_datetime(dt),
                    "symbol": sym_map.get(ysym, ysym),
                    "company_name": name_map.get(ysym, ""),
                    "Open": float(row["Open"]),
                    "High": float(row["High"]),
                })

    prices = pd.DataFrame.from_records(records, columns=["date", "symbol", "company_name", "Open", "High"])
    # Ensure types and ordering
    prices.sort_values(["date", "symbol"], inplace=True, kind="mergesort")
    prices.reset_index(drop=True, inplace=True)
    return prices

def _last_trading_days_from_prices(prices_df: pd.DataFrame, lookback_days: int) -> pd.DatetimeIndex:
    """
    Get the last N unique trading dates present in the prices DataFrame.
    """
    uds = pd.DatetimeIndex(sorted(prices_df["date"].dt.normalize().unique()))
    return uds[-lookback_days:] if len(uds) >= lookback_days else uds

def find_open_to_high_spikes(
    prices_df: pd.DataFrame,
    pct_threshold: float = 0.05,   # 5%
    lookback_days: int = 100
) -> pd.DataFrame:
    """
    Pure-analysis step (no downloads). Assumes prices_df has columns:
        date, symbol, company_name, Open, High

    Returns DataFrame:
        date, symbol, company_name, pct_increase  (pct_increase in %)
    """
    assert {"date", "symbol", "company_name", "Open", "High"}.issubset(prices_df.columns), \
        "prices_df missing required columns."

    # Limit to last N trading days *present in the df*
    last_days = _last_trading_days_from_prices(prices_df, lookback_days)
    sub = prices_df[prices_df["date"].dt.normalize().isin(last_days)].copy()
    if sub.empty:
        return pd.DataFrame(columns=["date", "symbol", "company_name", "pct_increase"])

    sub = sub[(sub["Open"] > 0) & sub["High"].notna()].copy()
    sub["pct_increase"] = (sub["High"] - sub["Open"]) / sub["Open"]

    hits = sub[sub["pct_increase"] >= pct_threshold].copy()
    if hits.empty:
        return pd.DataFrame(columns=["date", "symbol", "company_name", "pct_increase"])

    hits["pct_increase"] = (hits["pct_increase"] * 100.0).round(4)
    out = hits.loc[:, ["date", "symbol", "company_name", "pct_increase"]].copy()
    out.sort_values(by=["date", "pct_increase"], ascending=[True, False], inplace=True, kind="mergesort")
    out.reset_index(drop=True, inplace=True)
    return out

# -----------------------------
# Example usage
# -----------------------------
# if __name__ == "__main__":
#     # 1) Load S&P 500 constituents once
#     spx = load_sp500_constituents()

#     # 2) Download price history ONCE (choose enough days for your largest intended lookback)
#     #    e.g., if you'll try lookback_days up to 150, use period_days >= 150 + 60
#     prices = download_sp500_ohlc(spx, period_days=240, show_progress=False)

#     # (Optional) Save to CSV to reuse later without re-downloading:
#     # prices.to_csv("sp500_prices_open_high.csv", index=False)

#     # (Optional) Later, you can load from CSV instead of downloading:
#     # prices = pd.read_csv("sp500_prices_open_high.csv", parse_dates=["date"])

#     # 3) Run analysis many times WITHOUT re-downloading:
#     df_5pct_100d = find_open_to_high_spikes(prices, pct_threshold=0.05, lookback_days=100)
#     print(df_5pct_100d.head(20))

#     # Try different thresholds/lookbacks with the SAME in-memory DataFrame:
#     # df_7pct_80d = find_open_to_high_spikes(prices, pct_threshold=0.07, lookback_days=80)
#     # df_3pct_150d = find_open_to_high_spikes(prices, pct_threshold=0.03, lookback_days=150)




In [11]:
# requirements (pick what you use):
#   pip install requests feedparser python-dateutil pytz
#   # Optional if using Finnhub:
#   # pip install requests
#
# Usage preview (after your spikes df):
#   news_df = fetch_prev_day_news_for_hits(
#       hits_df=df_5pct_100d,           # columns: date, symbol, company_name, pct_increase
#       provider="auto",                 # "auto" | "finnhub" | "googlenews"
#       tz_str="America/New_York"
#   )
#   print(news_df.head())

import os
import re
import time
import json
import hashlib
import logging
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

import requests
import pandas as pd
import feedparser
from dateutil import tz
from datetime import datetime, timedelta

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("news_fetch")

# -----------------------------
# Utilities
# -----------------------------
FIN_KEYWORDS = [
    # earnings & outlook
    "earnings", "results", "q1", "q2", "q3", "q4", "quarter", "guidance", "forecast",
    "revenue", "sales", "profit", "eps", "net income", "operating income", "margin",
    # corporate actions / capital
    "dividend", "buyback", "repurchase", "stock split", "secondary offering", "ipo",
    "spinoff", "spin-off", "spac",
    # m&a & partnerships
    "acquisition", "acquire", "merger", "merge", "takeover", "deal", "partnership",
    # ratings & coverage
    "upgrade", "downgrade", "price target", "initiates coverage", "maintains",
    # operations / strategy
    "layoff", "restructuring", "guidance", "product launch", "recall", "contract",
    "order", "production", "factory", "plant", "strike", "union",
    # regulatory & legal
    "sec", "complaint", "investigation", "probe", "lawsuit", "settlement",
    "approval", "antitrust", "regulator", "fda", "fcc",
    # risk / distress
    "bankruptcy", "chapter 11", "default", "liquidity", "restatement", "data breach",
]

RE_WORD = re.compile(r"[A-Za-z0-9#\-\$]+")

def _clean_text(s: str) -> str:
    return " ".join(RE_WORD.findall((s or "").lower()))

def prev_calendar_day(d: pd.Timestamp, tz_str: str = "America/New_York") -> Tuple[datetime, datetime]:
    """
    For a trading date d (naive or tz-aware), return previous calendar day's [start,end] in the given timezone.
    """
    if isinstance(d, pd.Timestamp):
        dt_utc = d.tz_localize("UTC") if d.tzinfo is None else d.tz_convert("UTC")
        local = dt_utc.tz_convert(tz_str)
        local_prev = (local - timedelta(days=1)).date()
    else:
        local_prev = d.date() - timedelta(days=1)
    tzinfo = tz.gettz(tz_str)
    start = datetime(local_prev.year, local_prev.month, local_prev.day, 0, 0, 0, tzinfo=tzinfo)
    end   = datetime(local_prev.year, local_prev.month, local_prev.day, 23, 59, 59, tzinfo=tzinfo)
    # Normalize to naive UTC for providers that want UTC or ISO8601 with Z
    return (start.astimezone(tz.UTC).replace(tzinfo=None),
            end.astimezone(tz.UTC).replace(tzinfo=None))

def strip_legal_suffixes(name: str) -> str:
    """
    Remove common legal suffixes to broaden matches (Inc, Corp, plc, Ltd, Company, Class A, etc.)
    """
    if not name:
        return name
    s = re.sub(r",?\s+(inc\.?|corp\.?|corporation|co\.?|company|plc|ltd\.?|nv|sa|ag|class\s+[a-z])\b\.?", "", name, flags=re.I)
    return " ".join(s.split())

def build_company_terms(symbol: str, company_name: str) -> List[str]:
    """
    Return a set of query terms (ticker + cleaned company name + well-known alias if any).
    """
    base = strip_legal_suffixes(company_name)
    terms = {symbol.upper(), base}
    # minimal alias map for common rebrands
    ALIASES = {
        "META PLATFORMS": ["Facebook", "FB", "Meta"],
        "ALPHABET": ["Google", "GOOGL"],
        "PARAMOUNT GLOBAL": ["ViacomCBS"],
        "KENVUE": ["Johnson & Johnson consumer health"],
        "VERIZON COMMUNICATIONS": ["Verizon"],
        "AT&T": ["ATT", "AT&T Inc"],
        "BLOCK": ["Square"],
    }
    key = base.upper()
    if key in ALIASES:
        terms.update(ALIASES[key])
    return list(terms)

def financial_relevance_score(title: str, summary: str = "") -> int:
    """
    Simple keyword score. Increase threshold if you want stricter filtering.
    """
    text = _clean_text(f"{title} {summary}")
    score = sum(1 for kw in FIN_KEYWORDS if kw in text)
    return score

# -----------------------------
# Provider: Finnhub (preferred if available)
# -----------------------------
def fetch_finnhub_news(symbol: str, start_dt_utc: datetime, end_dt_utc: datetime, token: str) -> List[dict]:
    """
    Finnhub company-news endpoint (UTC date range).
    Docs: https://finnhub.io/docs/api/company-news
    """
    # Finnhub expects YYYY-MM-DD (UTC)
    frm = start_dt_utc.date().isoformat()
    to  = end_dt_utc.date().isoformat()
    url = "https://finnhub.io/api/v1/company-news"
    params = {"symbol": symbol.upper(), "from": frm, "to": to, "token": token}
    r = requests.get(url, params=params, timeout=20)
    if r.status_code != 200:
        logger.warning("Finnhub error %s for %s %s..%s", r.status_code, symbol, frm, to)
        return []
    items = r.json() or []
    # Normalize
    out = []
    for it in items:
        # it fields: category, datetime (unix), headline, id, image, related, source, summary, url
        pub_ts = datetime.utcfromtimestamp(it.get("datetime", 0))
        out.append({
            "title": it.get("headline", ""),
            "summary": it.get("summary", ""),
            "url": it.get("url", ""),
            "source": it.get("source", ""),
            "published_utc": pub_ts,
            "provider": "finnhub",
        })
    return out

# -----------------------------
# Provider: Google News RSS (fallback, no key)
# -----------------------------
def fetch_google_news(company_terms: List[str], start_dt_utc: datetime, end_dt_utc: datetime) -> List[dict]:
    """
    Query Google News RSS for each company term, then filter by UTC day window.
    No key needed, best-effort. Respect rate limits.
    """
    BASE = "https://news.google.com/rss/search"
    results = {}
    for term in company_terms:
        # Construct query: exact phrase OR ticker; fetch recent ("when:7d"), filter in code
        q = f'"{term}" when:7d'
        params = {"q": q, "hl": "en-US", "gl": "US", "ceid": "US:en"}
        resp = requests.get(BASE, params=params, timeout=20)
        time.sleep(0.3)  # be polite
        feed = feedparser.parse(resp.text)
        for entry in feed.entries:
            # published_parsed is time.struct_time in local tz? Treat as UTC-like, then parse from 'published'
            pub = None
            if hasattr(entry, "published_parsed") and entry.published_parsed:
                pub = datetime(*entry.published_parsed[:6])
            elif hasattr(entry, "updated_parsed") and entry.updated_parsed:
                pub = datetime(*entry.updated_parsed[:6])
            title = entry.title if hasattr(entry, "title") else ""
            summary = entry.summary if hasattr(entry, "summary") else ""
            link = entry.link if hasattr(entry, "link") else ""
            source = entry.source.title if hasattr(entry, "source") and hasattr(entry.source, "title") else "GoogleNews"

            if pub is None:
                continue

            # Keep only those within [start_dt_utc, end_dt_utc] (approximate; RSS timezones may vary)
            if not (start_dt_utc <= pub <= end_dt_utc):
                continue

            # Deduplicate by URL
            key = hashlib.md5(link.encode("utf-8")).hexdigest()
            if key not in results:
                results[key] = {
                    "title": title,
                    "summary": summary,
                    "url": link,
                    "source": source,
                    "published_utc": pub,
                    "provider": "googlenews",
                }
    return list(results.values())

# -----------------------------
# Coordinator with caching
# -----------------------------
@dataclass
class NewsConfig:
    provider: str = "auto"          # "auto" | "finnhub" | "googlenews"
    tz_str: str = "America/New_York"
    min_relevance: int = 1          # increase to 2 for stricter filtering
    require_company_mention: bool = True  # require company name (or alias) in title/summary
    sleep_between_calls: float = 0.0      # you can increase for rate limiting

def fetch_prev_day_news_for_hits(
    hits_df: pd.DataFrame,
    provider: str = "auto",
    tz_str: str = "America/New_York",
    min_relevance: int = 1,
    require_company_mention: bool = True,
) -> pd.DataFrame:
    """
    For each row in hits_df (date, symbol, company_name), fetch previous calendar day's news articles
    that are financially relevant. Returns a dataframe:

    columns: trading_date, symbol, company_name, pct_increase,
             news_published_utc, source, title, url, relevance_score, provider
    """
    assert {"date", "symbol", "company_name"}.issubset(hits_df.columns), \
        "hits_df must have columns: date, symbol, company_name"

    cfg = NewsConfig(provider=provider, tz_str=tz_str, min_relevance=min_relevance,
                     require_company_mention=require_company_mention)

    # Choose provider automatically
    finnhub_token = os.environ.get("FINNHUB_TOKEN", "").strip()
    effective_provider = cfg.provider
    if effective_provider == "auto":
        effective_provider = "finnhub" if finnhub_token else "googlenews"
    logger.info("Using news provider: %s", effective_provider)

    # Cache to avoid re-calling same (symbol, prev_day) during this run
    cache: Dict[Tuple[str, str], List[dict]] = {}

    rows = []
    # Grouping avoids repeated calls when same symbol appears with multiple spike dates
    for (sym, comp), grp in hits_df.groupby(["symbol", "company_name"]):
        terms = build_company_terms(sym, comp)
        for trading_date, sub in grp.groupby("date"):
            trading_ts = pd.Timestamp(trading_date)
            start_utc, end_utc = prev_calendar_day(trading_ts, tz_str=cfg.tz_str)
            cache_key = (sym.upper(), start_utc.date().isoformat())

            if cache_key in cache:
                articles = cache[cache_key]
            else:
                if effective_provider == "finnhub" and finnhub_token:
                    articles = fetch_finnhub_news(sym, start_utc, end_utc, finnhub_token)
                else:
                    articles = fetch_google_news(terms, start_utc, end_utc)
                cache[cache_key] = articles
                if cfg.sleep_between_calls > 0:
                    time.sleep(cfg.sleep_between_calls)

            # filter for financial relevance
            for a in articles:
                title = a.get("title", "")
                summary = a.get("summary", "")
                url = a.get("url", "")
                source = a.get("source", "")
                pub = a.get("published_utc", None)
                provider_name = a.get("provider", effective_provider)

                if not title or not url or pub is None:
                    continue

                score = financial_relevance_score(title, summary)
                if score < cfg.min_relevance:
                    continue

                if cfg.require_company_mention:
                    text = _clean_text(f"{title} {summary}")
                    # require at least one company term appears (loosely)
                    if not any(_clean_text(t) in text for t in terms if _clean_text(t)):
                        continue

                # Add a row per spike (symbol,date) so schema matches one-to-many
                for _, spike_row in sub.iterrows():
                    rows.append({
                        "trading_date": pd.to_datetime(spike_row["date"]).date(),
                        "symbol": sym,
                        "company_name": comp,
                        "pct_increase": float(spike_row.get("pct_increase", float("nan"))),
                        "news_published_utc": pd.to_datetime(pub),
                        "source": source,
                        "title": title,
                        "url": url,
                        "relevance_score": int(score),
                        "provider": provider_name,
                    })

    news_df = pd.DataFrame(rows, columns=[
        "trading_date", "symbol", "company_name", "pct_increase",
        "news_published_utc", "source", "title", "url", "relevance_score", "provider"
    ])
    news_df.sort_values(["trading_date", "symbol", "news_published_utc"], inplace=True)
    news_df.reset_index(drop=True, inplace=True)
    return news_df


In [3]:
# 1) Load S&P 500 constituents once
spx = load_sp500_constituents()

In [4]:
# 2) Download price history ONCE (choose enough days for your largest intended lookback)
#    e.g., if you'll try lookback_days up to 150, use period_days >= 150 + 60
prices = download_sp500_ohlc(spx, period_days=240, show_progress=False)


# (Optional) Save to CSV to reuse later without re-downloading:
# prices.to_csv("sp500_prices_open_high.csv", index=False)

# (Optional) Later, you can load from CSV instead of downloading:
# prices = pd.read_csv("sp500_prices_open_high.csv", parse_dates=["date"])


In [35]:

# 3) Run analysis many times WITHOUT re-downloading:
#df_5pct_100d = find_open_to_high_spikes(prices, pct_threshold=0.05, lookback_days=100)
#print(df_5pct_100d.head(20))

# Try different thresholds/lookbacks with the SAME in-memory DataFrame:
df_7pct_30d = find_open_to_high_spikes(prices, pct_threshold=0.07, lookback_days=30)
# df_3pct_150d = find_open_to_high_spikes(prices, pct_threshold=0.03, lookback_days=150)


In [36]:
df_7pct_30d.shape, df_7pct_30d['date'].nunique()

((39, 4), 21)

In [37]:
df_7pct_30d.tail(50)

,date,symbol,company_name,pct_increase
0,2025-07-08,MRNA,Moderna,11.8684
1,2025-07-08,ALB,Albemarle Corporation,9.2988
2,2025-07-08,FCX,Freeport-McMoRan,7.5777
3,2025-07-08,INTC,Intel,7.2588
4,2025-07-08,APA,APA Corporation,7.0175
5,2025-07-09,PTC,PTC Inc.,18.8933
6,2025-07-09,ENPH,Enphase Energy,8.9218
7,2025-07-10,UAL,United Airlines Holdings,7.3171
8,2025-07-18,IVZ,Invesco,12.0112
9,2025-07-22,IQV,IQVIA,10.9730


In [13]:
# After you build your spikes dataframe (e.g., df_5pct_100d):
news_df = fetch_prev_day_news_for_hits(
    hits_df=df_5pct_100d,
    provider="auto",                  # "auto" tries Finnhub, else Google News
    tz_str="America/New_York",
    min_relevance=1,                  # raise to 2 for stricter filtering
    require_company_mention=True
)

# Optionally save
# news_df.to_csv("spike_prevday_news.csv", index=False)


INFO:news_fetch:Using news provider: googlenews


In [24]:
news_df.loc[news_df.symbol == "ENPH"]

,trading_date,symbol,company_name,pct_increase,news_published_utc,source,title,url,relevance_score,provider
191,2025-08-15,ENPH,Enphase Energy,15.0875,2025-08-13 09:05:29,MarketBeat,"Enphase Energy, Inc. (NASDAQ:ENPH) Shares Sold...",https://news.google.com/rss/articles/CBMi3AFBV...,2,googlenews
192,2025-08-15,ENPH,Enphase Energy,15.0875,2025-08-13 13:30:47,Energy-Storage.News,'We will be good for the distant future': Enph...,https://news.google.com/rss/articles/CBMi0gFBV...,1,googlenews
193,2025-08-15,ENPH,Enphase Energy,15.0875,2025-08-13 17:07:00,TradingView,Enphase Energy Strengthens Battery Storage Foo...,https://news.google.com/rss/articles/CBMiuAFBV...,2,googlenews
194,2025-08-15,ENPH,Enphase Energy,15.0875,2025-08-13 18:18:07,Barchart.com,Enphase Energy Strengthens Battery Storage Foo...,https://news.google.com/rss/articles/CBMiqgFBV...,1,googlenews
195,2025-08-15,ENPH,Enphase Energy,15.0875,2025-08-13 18:33:32,AInvest,"Based on the current analyst ratings, is ENPH ...",https://news.google.com/rss/articles/CBMipgFBV...,1,googlenews


In [30]:
news_df[['trading_date', 'symbol']].drop_duplicates().shape

(38, 2)

In [31]:
df_5pct_100d.shape

(1375, 4)